In [1]:
# Dependencies
import tweepy
import numpy as np

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
# Target Search Term
target_terms = ("@SouthwestAir", "@AmericanAir", "@SpiritAirlines",
                "@Virginatlantic", "@Delta", "@AlaskaAir", "@KLM")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# Array to hold sentiment
sentiment_array = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(
            target, count=100, result_type="recent", max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Add each value to the appropriate array
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                
                # Set the new oldest_tweet value
                oldest_tweet = int(tweet["id_str"]) - 1

    # Store the Average Sentiments
    sentiment = {
        "User": target,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Negative": np.mean(negative_list),
        "Neutral": np.mean(neutral_list),
        "Tweet Count": len(compound_list)
    }

    # Print the Sentiments
    print(sentiment)
    print()

{'User': '@SouthwestAir', 'Compound': 0.19493321917808218, 'Positive': 0.14038356164383561, 'Neutral': 0.047453767123287673, 'Negative': 0.81215924657534244, 'Tweet Count': 584}

{'User': '@AmericanAir', 'Compound': -0.015134334763948501, 'Positive': 0.12478326180257511, 'Neutral': 0.094723175965665235, 'Negative': 0.78064592274678102, 'Tweet Count': 466}

{'User': '@SpiritAirlines', 'Compound': 0.071986641929499079, 'Positive': 0.10910018552875697, 'Neutral': 0.076291280148422999, 'Negative': 0.81458441558441552, 'Tweet Count': 539}

{'User': '@Virginatlantic', 'Compound': 0.18866035714285714, 'Positive': 0.13496785714285714, 'Neutral': 0.039151785714285715, 'Negative': 0.82588392857142856, 'Tweet Count': 560}

{'User': '@Delta', 'Compound': 0.22639082892416229, 'Positive': 0.14664550264550266, 'Neutral': 0.047546737213403877, 'Negative': 0.80581305114638446, 'Tweet Count': 567}

{'User': '@AlaskaAir', 'Compound': 0.24840355805243447, 'Positive': 0.15569288389513108, 'Neutral': 0.0269